In [ ]:
import numpy as np
import tensorflow as tf # for deep learning framework
from tensorflow.keras.layers import Input, Conv2D, Reshape # layers to build the cnn
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Generate QPSK Modulated Data
N = 10000  # Number of QPSK symbols
x = np.random.randint(0, 2, 2 * N)  # An array of random bits (0 or 1) of size 2*N

# QPSK Modulation
xmod = ((1 - 2*x[0::2]) + 1j * (1 - 2*x[1::2])) / np.sqrt(2)

# Define SNR range from 0 to 40 dB
snrdb_range = np.arange(0, 41, 1)
ber = np.zeros_like(snrdb_range, dtype=float)  # Initialize BER array

for idx, snrdb in enumerate(snrdb_range):
    snrlin = 10**(snrdb / 10)

    # Additive White Gaussian Noise (AWGN)
    noise = (np.random.randn(N) + 1j * np.random.randn(N)) / np.sqrt(2)

    # Received signal
    yrx = xmod + np.sqrt(1/snrlin) * noise

    # QPSK Demodulation
    ydemod = np.zeros(2 * N, dtype=int)
    ydemod[0::2] = np.real(yrx) < 0  # In-phase component
    ydemod[1::2] = np.imag(yrx) < 0  # Quadrature component

    # BER Calculation
    ber[idx] = np.sum(np.not_equal(ydemod, x)) / N

# CNN for QPSK Demodulation

# Reshape data for CNN input
xmod_real = np.real(xmod)
xmod_imag = np.imag(xmod)
xmod_noisy_real = np.real(yrx)
xmod_noisy_imag = np.imag(yrx)

# Concatenate real and imaginary parts
input_data = np.stack((xmod_noisy_real, xmod_noisy_imag), axis=-1)

# Reshape input_data to match CNN input shape
input_data = input_data.reshape((1, N, 2, 1))

#1: This represents the batch size (since we have a single batch of data).
#N: The number of QPSK symbols.
#2: The two features per symbol (real and imaginary parts).
#1: The channel dimension, which is used to fit the expected input format for Conv2D layers in Keras (channels last format).




# Concatenate real and imaginary parts for output
output_data = np.stack((xmod_real, xmod_imag), axis=-1)

# Define CNN architecture
input_layer = Input(shape=(N, 2, 1))  # Input layer for real and imaginary parts
conv1 = Conv2D(16, (3, 3), padding='same', activation='relu')(input_layer)
conv2 = Conv2D(32, (3, 3), padding='same', activation='relu')(conv1)
conv3 = Conv2D(2, (3, 3), padding='same')(conv2)



#16: Number of filters (feature maps) the convolutional layer will compute
#(3, 3): Size of the convolutional kernel (3x3 window)
#padding='same': Ensures that the output feature maps have the same spatial dimensions as the input by adding zero-padding where necessary

#o/p of conv1:
#1 is the batch size.
#N is the number of symbols
#2 is the second spatial dimension (features).
#16 is the number of output channels (filters).


# Reshape output of Conv2D layers to match output_data shape
reshape_layer = Reshape((N, 2))(conv3) ##############################################################################################

model = Model(inputs=input_layer, outputs=reshape_layer)

# Compile the model
#model.compile(optimizer=Adam(),loss='mean_squared_error')

# Train the model (you may need to adjust batch size and number of epochs based on your system's capabilities)
model.fit(input_data, output_data,
          epochs=10,
          batch_size=64,
          shuffle=True)

# Test the model
ypred = model.predict(input_data)

# Calculate BER for predicted symbols
ypred_real = np.squeeze(ypred[:, :, 0])
ypred_imag = np.squeeze(ypred[:, :, 1])

ypred_demod = np.zeros(2 * N, dtype=int)
ypred_demod[0::2] = ypred_real.flatten() < 0  # In-phase component
ypred_demod[1::2] = ypred_imag.flatten() < 0  # Quadrature component

ber_cnn = np.sum(np.not_equal(ypred_demod, x)) / N

print(f'BER using CNN: {ber_cnn:.6f}')


ValueError: Exception encountered when calling layer "reshape_1" (type Reshape).

total size of new array must be unchanged, input_shape = [10000, 2, 2], output_shape = [1, 10000, 2]

Call arguments received by layer "reshape_1" (type Reshape):
  • inputs=tf.Tensor(shape=(None, 10000, 2, 2), dtype=float32)